# FasterRCNN with TLT 3.0

This notebook gets you started with training FasterRCNN with ResNet18 backbone using Transfer Learning Toolkit 3.0

 ## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

In [1]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variables with your own.")
%env GPU_INDEX=0
%env KEY=tlt

# Please define this local project directory that needs to be mapped to the TLT docker session.
%env LOCAL_PROJECT_DIR=/home/astr1x/Projects/FasterRCNN-TLT-Triton
os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "faster_rcnn"
)
%env USER_EXPERIMENT_DIR=/workspace/tlt-experiments/faster_rcnn
%env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/data
# The sample spec files are present in the same path as the downloaded samples.
# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tlt-samples/faster_rcnn
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tlt-experiments/faster_rcnn/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

Please replace the variables with your own.
env: GPU_INDEX=0
env: KEY=tlt
env: LOCAL_PROJECT_DIR=/home/astr1x/Projects/FasterRCNN-TLT-Triton
env: USER_EXPERIMENT_DIR=/workspace/tlt-experiments/faster_rcnn
env: DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/data
env: SPECS_DIR=/workspace/tlt-experiments/faster_rcnn/specs
total 16
-rw-rw-r-- 1 astr1x astr1x 4029 Feb 25 21:15 default_spec_resnet18_retrain_spec.txt
-rw-rw-r-- 1 astr1x astr1x 3662 Feb 25 21:15 default_spec_resnet18_grayscale.txt
-rw-rw-r-- 1 astr1x astr1x  308 Jun  2 16:03 frcnn_tfrecords_kitti_trainval.txt
-rw-rw-r-- 1 astr1x astr1x 3679 Jun  2 16:10 default_spec_resnet18.txt


The cell below maps the project directory on your local host to a workspace directory in the TLT docker instance, so that the data and the results are mapped from outside to inside of the docker instance.

In [2]:
# Mapping up the local directories to the TLT docker.
import json
import os
mounts_file = os.path.expanduser("~/.tlt_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tlt-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ],
    # set gpu index for tlt-converter
    "Envs": [
        {"variable": "CUDA_VISIBLE_DEVICES", "value": os.getenv("GPU_INDEX")},
    ],
    "DockerOptions":{
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [3]:
!cat ~/.tlt_mounts.json

{
    "Mounts": [
        {
            "source": "/home/astr1x/Projects/FasterRCNN-TLT-Triton",
            "destination": "/workspace/tlt-experiments"
        },
        {
            "source": "/home/astr1x/Projects/FasterRCNN-TLT-Triton/faster_rcnn/specs",
            "destination": "/workspace/tlt-experiments/faster_rcnn/specs"
        }
    ],
    "Envs": [
        {
            "variable": "CUDA_VISIBLE_DEVICES",
            "value": "0"
        }
    ],
    "DockerOptions": {
        "user": "1000:1000"
    }
}

 ## 1. Prepare dataset <a class="anchor" id="head-2"></a>

In [4]:
# unpack 
!unzip -u $LOCAL_DATA_DIR/coco_val_skimmed.zip -d $LOCAL_DATA_DIR

Archive:  /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/coco_val_skimmed.zip
   creating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/
   creating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000295713.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000537827.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000500478.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000395801.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000213255.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000172617.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000438955.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000325991

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000430073.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000440184.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000100624.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000140583.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000419408.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000365208.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000562448.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000399205.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000135902.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000210099.jpg  
  inflating: /home/astr1x/Proj

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000115885.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000313454.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000131556.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000324258.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000309964.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000359540.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000138979.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000510329.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000468501.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000151962.jpg  
  inflating: /home/astr1x/Proj

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000499622.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000327306.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000534827.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000533493.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000083531.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000240250.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000458109.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000186042.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000119233.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000011511.jpg  
  inflating: /home/astr1x/Proj

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000481390.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000463802.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000432898.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000530099.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000027186.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000414034.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000182155.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000160012.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000193926.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000025603.jpg  
  inflating: /home/astr1x/Proj

...

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000548246.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000008690.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000289594.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000291619.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000489764.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000125129.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000098287.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000261732.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000505573.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000210388.jpg  
  inflating: /home/astr1x/Proj

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000032901.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000386912.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000213035.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000353518.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000018491.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000488673.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000475064.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000394559.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000276024.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000154358.jpg  
  inflating: /home/astr1x/Proj

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000023272.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000490171.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000448263.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000429690.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000065288.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000504000.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000358923.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000458992.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000189475.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000394940.jpg  
  inflating: /home/astr1x/Proj

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000511384.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000378139.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000480021.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000289741.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000441491.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000315187.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000410510.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000498286.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000119995.jpg  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/images/000000080340.jpg  
  inflating: /home/astr1x/Proj

  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000038678.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000193245.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000353027.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000178982.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000546829.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000009378.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000377723.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000286660.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000070254.txt  
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/data/training/labels/000000144300.txt  
  inflating: /home/astr1x/Proj

In [5]:
# verify
import os

DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/images")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/labels")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "training/images")))
print("Number of images in the train/val set. {}".format(num_training_images))
print("Number of labels in the train/val set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

Number of images in the train/val set. 2945
Number of labels in the train/val set. 2945
Number of images in the test set. 2945


In [6]:
print("TFrecords conversion spec file for training")
!cat $LOCAL_SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt

TFrecords conversion spec file for training
kitti_config {
  root_directory_path: "/workspace/tlt-experiments/data/training"
  image_dir_name: "images"
  label_dir_name: "labels"
  image_extension: ".jpg"
  partition_mode: "random"
  num_partitions: 2
  val_split: 14
  num_shards: 10
}
image_directory_path: "/workspace/tlt-experiments/data/training"


In [7]:
# Creating a new directory for the output tfrecords dump.
!mkdir -p $LOCAL_EXPERIMENT_DIR/tfrecords
#KITTI trainval
!tlt faster_rcnn dataset_convert --gpu_index $GPU_INDEX -d $SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt \
                     -o $DATA_DOWNLOAD_DIR/tfrecords/kitti_trainval/kitti_trainval

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-9asklkwj because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.
2021-06-03 04:35:51,219 - iva.detectnet_v2.dataio.build_converter - INFO - Instantiating a kitti converter
2021-06-03 04:35:51,226 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Num images in
Train: 2533	Val: 412
2021-06-03 04:35:51,226 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Validation data in partition 0. Hence, while choosing the validationset during training choose validation_fold 0.
2021-06-03 04:35:51,227 - iva.detectnet_v2.dataio.dataset_converter_lib - INFO - Writing partition 0, shard 0

2021-06-03 04:35:51,227 - tensorflow - WARNING - From /home/vpraveen/.cache/daze

Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000309391.txt. 
Coordinates: x1 = 252, x2 = 324, y1: 103, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 1 of label_file /workspace/tlt-experiments/data/training/labels/000000309391.txt. 
Coordinates: x1 = 221, x2 = 280, y1: 141, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 2 of label_file /workspace/tlt-experiments/data/training/labels/000000309391.txt. 
Coordinates: x1 = 269, x2 = 356, y1: 626, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 3 of label_file /workspace/tlt-experiments/data/training/labels/000000309391.txt. 
Coordinates: x1 = 186, x2 = 217, y1: 446, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 4 of label_file /workspace/tlt-experiments/data/training/

Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000033104.txt. 
Coordinates: x1 = 27, x2 = 463, y1: 213, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 1 of label_file /workspace/tlt-experiments/data/training/labels/000000033104.txt. 
Coordinates: x1 = 35, x2 = 475, y1: 380, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 2 of label_file /workspace/tlt-experiments/data/training/labels/000000033104.txt. 
Coordinates: x1 = 21, x2 = 181, y1: 396, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 3 of label_file /workspace/tlt-experiments/data/training/labels/000000033104.txt. 
Coordinates: x1 = 0, x2 = 97, y1: 428, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 4 of label_file /workspace/tlt-experiments/data/training/labels

Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000030494.txt. 
Coordinates: x1 = 29, x2 = 222, y1: 286, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000189752.txt. 
Coordinates: x1 = 1, x2 = 107, y1: 174, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 1 of label_file /workspace/tlt-experiments/data/training/labels/000000189752.txt. 
Coordinates: x1 = 0, x2 = 114, y1: 97, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 2 of label_file /workspace/tlt-experiments/data/training/labels/000000189752.txt. 
Coordinates: x1 = 0, x2 = 240, y1: 638, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/0

Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000422706.txt. 
Coordinates: x1 = 17, x2 = 225, y1: 148, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 1 of label_file /workspace/tlt-experiments/data/training/labels/000000422706.txt. 
Coordinates: x1 = 5, x2 = 157, y1: 186, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 2 of label_file /workspace/tlt-experiments/data/training/labels/000000422706.txt. 
Coordinates: x1 = 141, x2 = 317, y1: 120, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 3 of label_file /workspace/tlt-experiments/data/training/labels/000000422706.txt. 
Coordinates: x1 = 266, x2 = 275, y1: 261, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/lab

...

Top left coordinate must be less than bottom right.Error in object 9 of label_file /workspace/tlt-experiments/data/training/labels/000000136633.txt. 
Coordinates: x1 = 264, x2 = 283, y1: 110, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 10 of label_file /workspace/tlt-experiments/data/training/labels/000000136633.txt. 
Coordinates: x1 = 256, x2 = 344, y1: 33, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 11 of label_file /workspace/tlt-experiments/data/training/labels/000000136633.txt. 
Coordinates: x1 = 253, x2 = 282, y1: 179, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000559543.txt. 
Coordinates: x1 = 120, x2 = 329, y1: 319, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training

Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000193181.txt. 
Coordinates: x1 = 110, x2 = 500, y1: 307, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 1 of label_file /workspace/tlt-experiments/data/training/labels/000000193181.txt. 
Coordinates: x1 = 325, x2 = 568, y1: 419, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 2 of label_file /workspace/tlt-experiments/data/training/labels/000000193181.txt. 
Coordinates: x1 = 22, x2 = 88, y1: 293, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 3 of label_file /workspace/tlt-experiments/data/training/labels/000000193181.txt. 
Coordinates: x1 = 58, x2 = 125, y1: 326, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 4 of label_file /workspace/tlt-experiments/data/training/lab

Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000161879.txt. 
Coordinates: x1 = 1, x2 = 342, y1: 236, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000155291.txt. 
Coordinates: x1 = 81, x2 = 350, y1: 595, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000350405.txt. 
Coordinates: x1 = 145, x2 = 192, y1: 67, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 1 of label_file /workspace/tlt-experiments/data/training/labels/000000350405.txt. 
Coordinates: x1 = 84, x2 = 247, y1: 398, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/label

Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000181666.txt. 
Coordinates: x1 = 165, x2 = 269, y1: 324, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 1 of label_file /workspace/tlt-experiments/data/training/labels/000000181666.txt. 
Coordinates: x1 = 189, x2 = 215, y1: 75, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 2 of label_file /workspace/tlt-experiments/data/training/labels/000000181666.txt. 
Coordinates: x1 = 182, x2 = 261, y1: 15, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/labels/000000060835.txt. 
Coordinates: x1 = 44, x2 = 425, y1: 508, y2: 0
Skipping this object
Top left coordinate must be less than bottom right.Error in object 0 of label_file /workspace/tlt-experiments/data/training/lab

2021-06-03 10:05:55,135 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


In [8]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/kitti_trainval

total 1560
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00000-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00001-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00002-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00003-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00004-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00005-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00006-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00007-of-00010
-rw-r--r-- 1 astr1x astr1x  21320 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00008-of-00010
-rw-r--r-- 1 astr1x astr1x  22360 Jun  3 10:05 kitti_trainval-fold-000-of-002-shard-00009-of-00010

 ## 2. Download pretrained model <a class="anchor" id="head-2"></a>

In [9]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_reg_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli --no-check-certificate
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_reg_linux.zip
--2021-06-03 10:06:18--  https://ngc.nvidia.com/downloads/ngccli_reg_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 99.86.20.59, 99.86.20.99, 99.86.20.73, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|99.86.20.59|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 25048710 (24M) [application/zip]
Saving to: ‘/home/astr1x/Projects/FasterRCNN-TLT-Triton/ngccli/ngccli_reg_linux.zip’

ngccli_reg_linux.zi 100%[===================>]  23.89M  7.44MB/s    in 3.3s    

2021-06-03 10:06:21 (7.27 MB/s) - ‘/home/astr1x/Projects/FasterRCNN-TLT-Triton/ngccli/ngccli_reg_linux.zip’ saved [25048710/25048710]

Archive:  /home/astr1x/Projects/FasterRCNN-TLT-Triton/ngccli/ngccli_reg_linux.zip
  inflating: /home/astr1x/Projects/FasterRCNN-TLT-Triton/ngccli/ngc  
 extracting: /home/astr1x/Projects/FasterRCNN-TLT-Triton/ngccli/ngc.md5  


In [10]:
!ngc registry model list "nvidia/tlt_pretrained_object_detection*"

zsh:1: no matches found: nvidia/tlt_pretrained_object_detection*


In [11]:
# Download model from NGC.
!ngc registry model download-version nvidia/tlt_pretrained_object_detection:resnet18

Downloaded 82.38 MB in 5m 43s, Download speed: 245.61 KB/s               
----------------------------------------------------
Transfer id: tlt_pretrained_object_detection_vresnet18 Download status: Completed.
Downloaded local path: /home/astr1x/Projects/FasterRCNN-TLT-Triton/faster_rcnn/tlt_pretrained_object_detection_vresnet18
Total files downloaded: 1 
Total downloaded size: 82.38 MB
Started at: 2021-06-03 10:06:27.025696
Completed at: 2021-06-03 10:12:10.502011
Duration taken: 5m 43s
----------------------------------------------------


In [12]:
# Copy weights to experiment directory.
!cp tlt_pretrained_object_detection_vresnet18/resnet_18.hdf5 $LOCAL_EXPERIMENT_DIR
!rm -rf tlt_pretrained_object_detection_vresnet18
!ls -rlt $LOCAL_EXPERIMENT_DIR

total 94668
drwxrwxr-x 2 astr1x astr1x     4096 Jun  2 16:03 tfrecords
drwxrwxr-x 2 astr1x astr1x     4096 Jun  3 09:22 specs
-rw-rw-r-- 1 astr1x astr1x    16796 Jun  3 10:06 faster_rcnn_qat.ipynb
-rw-rw-r-- 1 astr1x astr1x  3628450 Jun  3 10:10 faster_rcnn.ipynb
-rw------- 1 astr1x astr1x 93278448 Jun  3 10:12 resnet_18.hdf5


 ## 3. Run TLT training <a class="anchor" id="head-4"></a>

In [13]:
!sed -i 's/$KEY/'"$KEY/g" $LOCAL_SPECS_DIR/default_spec_resnet18.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18.txt

# Copyright (c) 2017-2020, NVIDIA CORPORATION.  All rights reserved.
random_seed: 42
enc_key: 'tlt'
verbose: True
model_config {
input_image_config {
image_type: RGB
image_channel_order: 'bgr'
size_height_width {
height: 384
width: 640
}
    image_channel_mean {
        key: 'b'
        value: 103.939
}
    image_channel_mean {
        key: 'g'
        value: 116.779
}
    image_channel_mean {
        key: 'r'
        value: 123.68
}
image_scaling_factor: 1.0
max_objects_num_per_image: 100
}
arch: "resnet:18"
anchor_box_config {
scale: 64.0
scale: 128.0
scale: 256.0
ratio: 1.0
ratio: 0.5
ratio: 2.0
}
freeze_bn: True
freeze_blocks: 0
freeze_blocks: 1
roi_mini_batch: 256
rpn_stride: 16
use_bias: False
roi_pooling_config {
pool_size: 7
pool_size_2x: False
}
all_projections: True
use_pooling:False
}
dataset_config {
  data_sources: {
    tfrecords_path: "/workspace/tlt-experiments/data/tfrecords/kitti_trainval/kitti_trainval*"
    image_directory_path: "/workspace/tlt-experiments/data/trai

In [14]:
!tlt faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-s4ieqqi8 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.

2021-06-03 04:42:19,556 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/horovod/tensorflow/__init__.py:117: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.


2021-06-03 04:42:19,556 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/horovod/tensorflow/__init__.py:143: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2021-06-03 04:42:19,642 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8


2021-06-03 04:42:21,508 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/faster_rcnn/data_loader/inputs_loader.py:221: The name tf.debugging.assert_less_equal is deprecated. Please use tf.compat.v1.debugging.assert_less_equal instead.


2021-06-03 04:42:21,907 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.


2021-06-03 04:42:22,305 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/faster_rcnn/layers/utils.py:76: The name tf.debugging.assert_less is deprecated. Please use tf.compat.v1.debugging.assert_less instead.


2021-06-03 04:42:23,230 [WARN


2021-06-03 04:45:31,271 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/faster_rcnn/layers/utils.py:64: The name tf.unsorted_segment_min is deprecated. Please use tf.math.unsorted_segment_min instead.

2021-06-03 04:45:38,360 [INFO] __main__: Building validation dataset...
2021-06-03 04:45:40,909 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: Serial augmentation enabled = False
2021-06-03 04:45:40,909 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: Pseudo sharding enabled = False
2021-06-03 04:45:40,909 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: Max Image Dimensions (all sources): (0, 0)
2021-06-03 04:45:40,909 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: number of cpus: 16, io threads: 32, compute threads: 16, buffered batches: 4
2021-06-03 04:


2021-06-03 04:57:11,491 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.


2021-06-03 04:57:11,631 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

Epoch 1/12
[1dac65716b01:00060] *** Process received signal ***
[1dac65716b01:00060] Signal: Segmentation fault (11)
[1dac65716b01:00060] Signal code: Address not mapped (1)
[1dac65716b01:00060] Failing at address: 0x10
[1dac65716b01:00060] [ 0] /lib/x86_64-linux-gnu/libc.so.6(+0x3f040)[0x7f61712eb040]
[1dac65716b01:00060] [ 1] /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/_pywrap_tensorflow_internal.so(_ZN10tensorflow8BinaryOpIN5Eigen9GpuDeviceENS_7functor3mulIfEEE7ComputeEPNS_15OpKernelContextE+0x100)[0x7f60fc7b4b30]
[1dac65716b01:00060] [ 2] /usr/local/

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
print("For multi-GPU, please uncomment and run this instead. Change --gpus  and --gpu_index based on your machine.")
# !tlt faster_rcnn train -e $SPECS_DIR/default_spec_resnet18.txt \
#                    --gpus 2 \
#                    --gpu_index 1 2

In [ ]:
print("For resume training from checkpoint, please uncomment and run this instead. Change/Add the 'resume_from_model' field in the spec file.")
# !tlt faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

In [ ]:
print("For Automatic Mixed Precision(AMP) training, please uncomment and run this. Make sure you use the Volta or above GPU arch to enable AMP.")
# !tlt faster_rcnn train --gpu_index $GPU_INDEX --use_amp -e $SPECS_DIR/default_spec_resnet18.txt

 ## 4. Evaluate trained models <a class="anchor" id="head-5"></a>

In [ ]:
!tlt faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

 ## 5. Prune trained models <a class="anchor" id="head-6"></a>
 * Specify pre-trained model
 * Equalization criterion
 * Threshold for pruning
 * A key to save and load the model
 * Output directory to store the model
 
Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is depend on the dataset. A `pth` value below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!tlt faster_rcnn prune --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18.epoch12.tlt \
           -o $USER_EXPERIMENT_DIR/model_1_pruned.tlt  \
           -eq union  \
           -pth 0.2 \
           -k $KEY

In [ ]:
!ls -lht $LOCAL_EXPERIMENT_DIR

 ## 6. Retrain pruned models <a class="anchor" id="head-7"></a>
 * Model needs to be re-trained to bring back accuracy after pruning
 * Specify re-training specification

In [ ]:
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!sed -i 's/$KEY/'"$KEY/g" $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tlt faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Listing the newly retrained model.
!ls -lht $LOCAL_EXPERIMENT_DIR

 ## 7. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [ ]:
!tlt faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ## 8. Visualize inferences <a class="anchor" id="head-9"></a>
 In this section, we run the inference tool to generate inferences on the trained models.

In [ ]:
# Running inference for detection on n images
# Please go to $LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain to see the visualizations.
!tlt faster_rcnn inference --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

The `inference` tool produces two outputs. 
1. Overlain images in `$LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain`
2. Frame by frame bbox labels in kitti format located in `$LOCAL_EXPERIMENT_DIR/inference_dump_labels_retrain`

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ## 9. Deploy! <a class="anchor" id="head-10"></a>

In [ ]:
# Export in FP32 mode.
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.etlt; fi
!tlt faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY

In [ ]:
# Export in FP16 mode.
# Note that the .etlt model in FP16 mode is the same as in FP32 mode.
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt; fi
!tlt faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --data_type fp16

In [ ]:
# Export in INT8 mode(generate calibration cache file).
# Note that the .etlt model in INT8 mode is the same as in FP32 mode.
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt; fi
!tlt faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --data_type int8 \
                        --batch_size 8 \
                        --batches 10 \
                        --cal_cache_file $USER_EXPERIMENT_DIR/cal.bin

In [ ]:
# Converting to TensorRT engine(FP32) is omitted here as this is trivial.
# Convert to TensorRT engine(FP16).
# Make sure your GPU type supports the FP16 data type before running this cell.
!tlt tlt-converter -k $KEY  \
               -d 3,384,1248 \
               -o NMS \
               -e $USER_EXPERIMENT_DIR/trt.fp16.engine \
               -m 4 \
               -t fp16 \
               -i nchw \
               $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_fp16.etlt

In [ ]:
# Convert to TensorRT engine(INT8).
# Make sure your GPU type supports the INT8 data type before running this cell.
!tlt tlt-converter -k $KEY  \
               -d 3,384,1248 \
               -o NMS \
               -c $USER_EXPERIMENT_DIR/cal.bin \
               -e $USER_EXPERIMENT_DIR/trt.int8.engine \
               -b 8 \
               -m 4 \
               -t int8 \
               -i nchw \
               $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8.etlt

In [ ]:
print('Exported model and converted TensorRT engine:')
print('------------')
!ls -lht $LOCAL_EXPERIMENT_DIR